In [1]:
# Imports
import sys
import numpy as np

%cd Q:\sachuriga\Sachuriga_Python\cnn-ripple-main\src/notebooks
sys.path.insert(1, 'Q:\sachuriga\Sachuriga_Python\cnn-ripple-main\src/cnn/')

Q:\sachuriga\Sachuriga_Python\cnn-ripple-main\src\notebooks


In [2]:
import os
sys.path.insert(1, r'Q:\sachuriga\Sachuriga_Python\cnn-ripple-main/figshare')
from figshare import Figshare

fshare = Figshare()

article_id = 14960085 # This is the ID of the data repository

datapath = "figshare_%d"%(article_id)

if os.path.isdir(datapath):
    print("Data already exists. Moving on.")
else:
    print("Downloading data... Please wait")
    fshare.retrieve_files_from_article(article_id)
    print("Data downloaded!")

Data downloaded!


In [4]:
'''''''''''''''''
Load data from the figshare files
'''''''''''''''''
article_id = 14960085
datapath = "figshare_%d"%(article_id)
from load_data import load_data

shank = 1

# Load data
print("Loading data...", end=" ")
loaded_data, fs = load_data(path=datapath, shank=shank)
print("Done!")

print("Shape of loaded data: ", np.shape(loaded_data))

Loading data... Done!
Shape of loaded data:  (22326272, 8)


In [5]:
from load_data import z_score_normalization, downsample_data
# Downsample data
downsampled_fs = 1250
print("Downsampling data from %d Hz to %d Hz..."%(fs, downsampled_fs), end=" ")
data = downsample_data(loaded_data, fs, downsampled_fs)
print("Done!")

# Normalize it with z-score
print("Normalizing data...", end=" ")
data = z_score_normalization(data)
print("Done!")

print("Shape of loaded data after downsampling and z-score: ", np.shape(data))

Downsampling data from 30000 Hz to 1250 Hz... Done!
Normalizing data... Done!
Shape of loaded data after downsampling and z-score:  (930261, 8)


In [6]:
overlapping = True
window_size = 0.0128

print("Generating windows...", end=" ")
if overlapping:
    from load_data import generate_overlapping_windows
    
    stride = 0.0064

    # Separate the data into 12.8ms windows with 6.4ms overlapping
    X = generate_overlapping_windows(data, window_size, stride, downsampled_fs)
else:
    stride = window_size
    X = np.expand_dims(data, 0)
print("Done!")

Generating windows... Done!


In [9]:
import tensorflow.keras.backend as K
import tensorflow.keras as kr

print("Loading CNN model...", end=" ")
optimizer = kr.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
model = kr.models.load_model("../../model", compile=False)
model.compile(loss="binary_crossentropy", optimizer=optimizer)

print("Done!")

ImportError: Traceback (most recent call last):
  File "c:\Users\sachur\AppData\Local\anaconda3\envs\test1\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
print("Detecting ripples...", end=" ")
predictions = model.predict(X, verbose=True)
print("Done!")

In [ ]:
from format_predictions import get_predictions_indexes

# This threshold can be changed
threshold = 0.7

print("Getting detected ripples indexes and times...", end=" ")
pred_indexes = get_predictions_indexes(data, predictions, window_size=window_size, stride=stride, fs=downsampled_fs, threshold=threshold)

pred_times = pred_indexes / downsampled_fs
print("Done!")

In [ ]:
#@markdown This is an interactive plot of the loaded data, where detected ripples are shown in blue. Data is displayed in chunks of 1 seconds and you can **move forward, backwards or jump to an specific second** using the control bar at the bottom.\
#@markdown \
#@markdown Run this cell to load the plotting method. Execute the **following** cell to use the method.

import matplotlib.pyplot as plt

def plot_ripples(k):
  data_size = data.shape[0]
  data_dur = data_size / downsampled_fs
  times = np.arange(data_size) / downsampled_fs

  if k >= times[-1]:
    printf("Data is only %ds long!"%(times[-1]))
    return
  elif k < 0:
    print("Please introduce a valid integer.")
    return

  ini_idx = int(k * downsampled_fs)
  end_idx = np.minimum(int((k+1) * downsampled_fs), data_size-1)


  pos_mat = list(range(data.shape[1]-1, -1, -1)) * np.ones((end_idx-ini_idx, data.shape[1]))

  fig = plt.figure(figsize=(9.75,5))
  ax = fig.add_subplot(1, 1, 1)
  ax.set_ylim(-3, 9)
  ax.margins(x=0)
  plt.tight_layout()
  plt.xlabel("Time (s)")


  lines = ax.plot(times[ini_idx:end_idx], data[ini_idx:end_idx, :]*1/np.max(data[ini_idx:end_idx, :], axis=0) + pos_mat, color='k', linewidth=1)

  fills = []
  for pred in pred_indexes:
      if (pred[0] >= ini_idx and pred[0] <= end_idx) or (pred[1] >= ini_idx and pred[1] <= end_idx):
          rip_ini = (pred[0]) / downsampled_fs
          rip_end = (pred[1]) / downsampled_fs
          fill = ax.fill_between([rip_ini, rip_end], [-3, -3], [9, 9], color="tab:blue", alpha=0.3)
          fills.append(fill)

  plt.show()

print("Loaded!")

In [ ]:
#@title Time (in seconds) { run: "auto", vertical-output: true, display-mode: "form" }
second =  0#@param {type:"integer"}

plot_ripples(second)